# Emotion detection using CNN 
## B) Testing model with camera

In [1]:
import os
import cv2
import numpy as np
import keras
from keras.models import model_from_json
from keras.preprocessing import image
import sklearn.externals
import joblib
import h5py
from tkinter import *
import tensorflow as tf
from PIL import ImageTk, Image
import time

#### 1) Loading the model with H5PY & the face detectcion model

In [2]:
model = keras.models.load_model('model_emotion_detection.h5')
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#### 2) Program performing emotion detection

In [ ]:
# Creer une fenetre
window = Tk()
 
# Personnaliser la fenetre
window.title("Emotion recognition")
window.geometry("1200x720")
window.config(background = "darkblue")
 
# Premier texte
label_title = Label(window, text = "Present your face in front of the webcam and click on Start button", font = ("Arial", 25), bg = "darkblue", fg = "white")
label_title.pack() #pour afficher le titre
 
# Second texte
label_subtitle = Label(window, text = "Here's your average emotion : ", font = ("Arial", 20), bg = "darkblue", fg = "white")
label_subtitle.pack(expand = YES) #pour afficher le titre

# Definition de variables qui vont nous aider pour le programme
global activation_sytem
activation_sytem = 0 

global nbr_iteration
nbr_iteration = 0

global del_label
del_label = 0

# Fonction d'activation de la webcam et du code de reconnaissance facial correspondant
def open_webcam():
    cap = cv2.VideoCapture(0)
    activation_sytem = 1
    nbr_iteration = 0
    predictions_sum = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
    
    # Variable del_label qui, une fois au dessus de 1, permettra d'éffacer le contenu des Labels de l'interface dans le but de refaire le test de reconnaissance
    global del_label
    if del_label > 0:
        global label_subsubtitle1
        label_subsubtitle1.destroy()
        global label_subsubtitle2
        label_subsubtitle2.destroy()
        global label_subsubtitle3
        label_subsubtitle3.destroy()
        global label_subsubtitle4
        label_subsubtitle4.destroy()
        global label_subsubtitle5
        label_subsubtitle5.destroy()
        global label_subsubtitle6
        label_subsubtitle6.destroy()
        global label_subsubtitle7
        label_subsubtitle7.destroy()
        global label_subtitle8
        label_subtitle8.destroy()
        
    if del_label == 0:
        del_label = 1
    
    # Partie de code lisant le contenu de la webcam et qui attribue une émotion à ce contenu
    while activation_sytem == 1:
        
        # Lecture de la webcam
        ret,image = cap.read()
        # convertion de l'image en niveau de gris
        converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Detection du visage sur l'image
        faces_detected = face_haar_cascade.detectMultiScale(converted_image,scaleFactor=1.3, minNeighbors=5)

        for (x,y,w,h) in faces_detected:
            
            # Dessin du rectangle autour du visage
            cv2.rectangle(image,(x,y), (x+w,y+h), (255,0,0))
            # Calibration de l'image en 48x48 pixels
            roi_gray = converted_image[y:y+w,x:x+h]
            roi_gray = cv2.resize(roi_gray,(48,48))
            
            # Transformation de l'image en un array
            image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
            image_pixels = np.expand_dims(image_pixels, axis = 0)
            image_pixels /= 255
            
            # Prédiction par le model
            predictions = model.predict(image_pixels) 
            max_index = np.argmax(predictions[0])
            
            # Attibution de l'émotion (en string) correspondant à la prédiction du model
            emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            emotion_prediction = emotion_detection[max_index]
            
            #calcul de la somme des probabilités correspondant à chaque émotion
            predictions_sum[0][0] = predictions_sum[0][0] + predictions[0][0]
            predictions_sum[0][1] = predictions_sum[0][1] + predictions[0][1]
            predictions_sum[0][2] = predictions_sum[0][2] + predictions[0][2]
            predictions_sum[0][3] = predictions_sum[0][3] + predictions[0][3]
            predictions_sum[0][4] = predictions_sum[0][4] + predictions[0][4]
            predictions_sum[0][5] = predictions_sum[0][5] + predictions[0][5]
            predictions_sum[0][6] = predictions_sum[0][6] + predictions[0][6]
            
            # Incrémentation de la variable pour connaître le nombre d'itération effectué et ensuite pouvoir faire une moyenne des émotions
            nbr_iteration = nbr_iteration + 1
            
            # Détermination de la police d'écriture
            font = cv2.FONT_HERSHEY_SIMPLEX
            # Emplacement et propriétés du text "Emotion" placé sur l'image
            cv2.putText(image, emotion_prediction, (int(x), int(y)), font, 3, (0, 0, 255), 2, cv2.LINE_4)
            
            # Affichage de l'image/vidéo
            cv2.imshow('Demo video', image)
    
           
            # Arrêt de la démonstration une fois avoir atteint le nombre de 60 itérations
            if cv2.waitKey(2) & nbr_iteration == 60:
                
                # Arret de l'utilisation de la webcam
                cap.release()
                
                # calcul de la moyenne des prédictions de chaque émotions sur l'entierté de la démonstration
                moyenne_angry = (predictions_sum[0][0]/nbr_iteration)*100
                mo_angry = round(moyenne_angry,1)
                
                moyenne_disgust = (predictions_sum[0][1]/nbr_iteration)*100
                mo_disgust = round(moyenne_disgust,1)
                
                moyenne_fear = (predictions_sum[0][2]/nbr_iteration)*100
                mo_fear = round(moyenne_fear,1)
                
                moyenne_happy = (predictions_sum[0][3]/nbr_iteration)*100
                mo_happy = round(moyenne_happy,1)
                
                moyenne_sad = (predictions_sum[0][4]/nbr_iteration)*100
                mo_sad = round(moyenne_sad,1)
                
                moyenne_surprise = (predictions_sum[0][5]/nbr_iteration)*100
                mo_surprise = round(moyenne_surprise,1)
                
                moyenne_neutral = (predictions_sum[0][6]/nbr_iteration)*100
                mo_neutral = round(moyenne_neutral,1)
                
                #Determination de l'émotion dominante
                tab_moyenne = [[mo_angry, mo_disgust, mo_fear, mo_happy, mo_sad, mo_surprise, mo_neutral]]
                max_moy = np.argmax(tab_moyenne[0])
                dominant = emotion_detection[max_moy]
                
                #Affichage de ces moyennes sur l'interface
                label_subsubtitle1 = Label(window, text = ('angry', mo_angry,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle2 = Label(window, text = ('disgust', mo_disgust,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle3 = Label(window, text = ('fear', mo_fear,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle4 = Label(window, text = ('happy', mo_happy,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle5 = Label(window, text = ('sad', mo_sad,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle6 = Label(window, text = ('surprise', mo_surprise,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                label_subsubtitle7 = Label(window, text = ('neutral', mo_neutral,'%') , font = ("Arial", 20), bg = "darkblue", fg = "white")
                
                label_subsubtitle1.pack(expand = YES) 
                label_subsubtitle2.pack(expand = YES) 
                label_subsubtitle3.pack(expand = YES)
                label_subsubtitle4.pack(expand = YES)
                label_subsubtitle5.pack(expand = YES)
                label_subsubtitle6.pack(expand = YES)
                label_subsubtitle7.pack(expand = YES)
                
                #Affichage de l'émotion dominante sur l'interface
                label_subtitle8 = Label(window, text = ('Dominant:', dominant) , font = ("Arial", 30), bg = "darkblue", fg = "red")
                label_subtitle8.pack(expand = YES) #pour afficher le titre
                
                # Fermeture de la fenêtre contenant la webcam
                cv2.destroyAllWindows()
                
                # Arrêt de la boucle for
                break
               
# Fonction Stop permettant la fermeture de la fenêtre d'interface
def stop():
    activation_sytem=0
    window.destroy()
        
#  Ajout des boutons Start et Close permettant donc de lancer la démonstration et de fermer la fenêtre d'interface
button_Stop = Button(window, text = "Close", font =("Arial", 20), bg = 'white', fg = 'darkblue', command = stop)
button_Stop.pack(side = BOTTOM, fill = X, expand = False)

button_Start = Button(window, text = "Start", font =("Arial", 20), bg = 'white', fg = 'darkblue', command = open_webcam)
button_Start.pack(side = BOTTOM, fill = X, expand = False)

# Affichage de l'interface
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\romai\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-3-290c30165af3>", line 63, in open_webcam
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\romai\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-3-290c30165af3>", line 63, in open_webcam
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvt

Source :
- https://analyticsindiamag.com/my-first-cnn-project-emotion-detection-using-convolutional-neural-network-with-tpu/

- https://www.youtube.com/watch?v=DtBu1u5aBsc&t=2363s